In [28]:
from pyspark.sql import SparkSession
import os
import sys
from pyspark.sql.functions import *

import findspark
findspark.init('C:/Spark/spark-3.4.2-bin-hadoop3/')

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable



from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.driver.memory", "10g").master('local[*]').appName('lab_spark')\
    .config('spark.jars.packages', 'org.apache.spark:spark-avro_2.12:3.4.2')\
    .config('spark.jars', 'C:\\Spark\\spark-3.4.2-bin-hadoop3\\jars\\postgresql-42.7.3.jar')\
    .getOrCreate()

In [29]:
data = spark.read.format("avro").load("orders_avro")

In [30]:
data.show()

+-------+--------------------+---------+-----------+--------+--------+------------+-------+----------+-----------+
|country|       facility_name|      lat|        lng|order_id|    time|        food|  price|      date|       city|
+-------+--------------------+---------+-----------+--------+--------+------------+-------+----------+-----------+
| Russia|         Супра Меоре|43.101581|131.9051689|   73208|01:02:09|     Tom yum| 346.23|2025-07-03|Vladivostok|
| Russia|         Супра Меоре|43.101581|131.9051689|   59817|01:19:49|     Tom yum| 346.23|2025-07-03|Vladivostok|
| Russia|         Супра Меоре|43.101581|131.9051689|   59817|01:19:49|       Pasta| 329.13|2025-07-03|Vladivostok|
| Russia|TOKYO Kawaii на С...|43.119274| 131.882176|   66084|04:31:58|         Pie| 482.98|2025-07-03|Vladivostok|
| Russia|TOKYO Kawaii на С...|43.119274| 131.882176|   66084|04:31:58|         Pie| 482.98|2025-07-03|Vladivostok|
| Russia|TOKYO Kawaii на С...|43.119274| 131.882176|   28648|05:20:32|   Hamburg

Создадим 3НФ

In [31]:
menu = data.select('food','price').distinct().withColumn('menu_id', monotonically_increasing_id())
restourants = data.select('city','facility_name','lat','lng').distinct().withColumn('rest_id', monotonically_increasing_id())
citys= data.select('city','country' ).distinct().withColumn('city_id', monotonically_increasing_id())
orders = data.select('food','date','time' ).distinct().withColumn('order_id', monotonically_increasing_id())


In [32]:
true_rest = restourants.join(citys, on='city').drop('city').drop('country')
true_rest.sort('rest_id').show()     




+--------------------+----------+-----------+-------+-------+
|       facility_name|       lat|        lng|rest_id|city_id|
+--------------------+----------+-----------+-------+-------+
|  Сказка. Еда и вино|57.6253632| 39.8749721|      0|      2|
|TOKYO Kawaii на С...| 43.119274| 131.882176|      1|      5|
|              Паштет|56.8405019| 60.6082934|      2|      1|
|              Брюгге|57.6230007| 39.8818897|      3|      2|
|    Sabor de la Vida|55.7679679|  37.558749|      4|      6|
|          Троекуровъ| 56.841769| 60.6620667|      5|      1|
|             Кремлин|55.7937397| 49.1099983|      6|      9|
|           Союз-rest|56.2422779| 43.8601695|      7|      4|
|       #СИБИРЬСИБИРЬ|55.0295233| 82.9049737|      8|      7|
|   Княгининский Двор|48.6948735| 44.4986003|      9|      3|
|     Любовь Пирогова| 55.712178|  37.620643|     10|      6|
|        Dans Le Noir|59.9448238| 30.2938052|     11|      8|
|         Puppen Haus|55.0261368| 82.9205541|     12|      7|
|       

In [33]:
true_orders = data.join(true_rest, on='facility_name').\
                join(menu, on='food').\
                drop('facility_name','city','country','food','price','lat','lng').distinct()

In [34]:
true_orders.show()

+--------+--------+----------+-------+-------+-------+
|order_id|    time|      date|rest_id|city_id|menu_id|
+--------+--------+----------+-------+-------+-------+
|   50380|05:43:11|2025-02-25|     17|      0|      1|
|   59211|06:11:21|2025-07-04|     17|      0|      3|
|   13572|06:42:42|2025-03-26|     17|      0|      4|
|   23951|23:16:57|2025-06-03|     17|      0|      2|
|   71095|19:43:40|2025-09-22|     17|      0|     18|
|   71095|19:43:40|2025-09-22|     17|      0|     16|
|   14450|08:22:40|2025-01-20|     17|      0|      3|
|   57337|17:32:51|2025-12-05|     17|      0|     20|
|   20813|17:49:02|2025-01-18|     17|      0|     10|
|   48098|08:21:58|2025-07-18|     17|      0|      6|
|   22923|10:38:20|2025-10-14|     17|      0|      8|
|   30067|10:50:46|2025-02-11|     17|      0|     18|
|   39395|15:58:44|2025-12-27|     17|      0|      0|
|   17833|18:20:39|2025-10-02|     17|      0|      5|
|   56735|08:34:34|2025-05-31|     17|      0|      1|
|   26811|

In [35]:
kwargs = {
    'properties':{
    "user": "postgres",
    "password": "12345",
    "driver": "org.postgresql.Driver"},
    'mode':'append',
    'url':'jdbc:postgresql://localhost:5432/task3',
}



true_orders.write.jdbc(table="orders",**kwargs )
citys.write.jdbc(table="citys",**kwargs)
menu.write.jdbc(table="menu",**kwargs)
true_rest.write.jdbc( table="restourant",**kwargs)